In [3]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:
from qdrant_client import QdrantClient, models 

/workspaces/llmzoomcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient("http://localhost:6333") # connect to local qdrant instance 

In [3]:
import requests 

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [5]:
from fastembed import TextEmbedding 
TextEmbedding.list_supported_models() # show models supported and their supported dimension, size and sources 

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [6]:
import json 
EMBEDDING_DIMENSIONALITY = 512 

for model in TextEmbedding.list_supported_models():
    if model['dim'] == EMBEDDING_DIMENSIONALITY: 
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [7]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [8]:
collection_name = "zoomcamp-rag"

client.create_collection(
    collection_name = collection_name, 
    vectors_config=models.VectorParams(
        size = EMBEDDING_DIMENSIONALITY,
        distance = models.Distance.COSINE 
    )
)

True

In [9]:
# create, embed, insert points into collection 
points = [] 
id = 0 

for course in documents_raw: 
    for doc in  course['documents']:
    
        point = models.PointStruct(
            id = id, 
            vector=models.Document(text=doc['text'],model = model_handle),  # embed locally 
            payload = {
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } # metadata 
        )
        points.append(point)
        id += 1 

In [10]:
# upsert points into collection 
client.upsert(
    collection_name = collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files: 100%|██████████| 5/5 [00:03<00:00,  1.34it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
# Visualise clustering 
# run similarity search to find the most relevant answer 
def search(query,limit=1):
    results = client.query_points(
        collection_name = collection_name,
        query=models.Document(
            text=query,
            model = model_handle
        ),
        limit = limit , 
        with_payload = True
    )
    return results 

In [13]:
import random 
course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece,indent=2))

{
  "text": "Use \u2018pandas.concat\u2019 function (https://pandas.pydata.org/docs/reference/api/pandas.concat.html) to combine two dataframes. To combine two numpy arrays use numpy.concatenate (https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html) function. So the code would be as follows:\ndf_train_combined = pd.concat([df_train, df_val])\ny_train = np.concatenate((y_train, y_val), axis=0)\n(George Chizhmak)",
  "section": "2. Machine Learning for Regression",
  "question": "How to combine train and validation datasets"
}


In [14]:
result = search(course_piece['question'])
result 

QueryResponse(points=[ScoredPoint(id=664, version=0, score=0.8582178, payload={'text': 'Filling in missing values using an entire dataset before splitting for training/testing/validation causes', 'section': '6. Decision Trees and Ensemble Learning', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [15]:
# run similarity search with filters 
client.create_payload_index(
    collection_name = collection_name,
    field_name = "course",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
# updated search 

def search_in_course(query, course = "mlops-zoomcamp", limit=1):
    results = client.query_points(
        collection_name = collection_name,
        query=models.Document(
            text=query,
            model = model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
    
        limit = limit , 
        with_payload = True
    )
    return results 

In [17]:
print(search_in_course("what if I submit homework late?","machine-learning-zoomcamp").points[0].payload['text'])

Depends on whether the form will still be open. If you're lucky and it's open, you can submit your homework and it will be evaluated. if closed - it's too late.
(Added by Rileen Sinha, based on answer by Alexey on Slack)


In [18]:
print(search_in_course("what if I submit homework late?","data-engineering-zoomcamp").points[0].payload['text'])

No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


# RAG 